# HlyB Atomistic All Trajectory Analysis

## MD Analysis

### Import Modules

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statistics as s
import MDAnalysis as mda

### Helper Modules

In [ ]:
# Dictionary which converts three letter code to one letter code
convert_to_one_letter_dic = {"ALA":"A", "ARG":"R", "ASN":"N", "ASP":"D", "CYS":"C", "GLU":"E", "GLN":"Q", "GLY":"G",\
                            "HSD":"H", "HIS": "H", "HSE":"H", "ILE": "I", "LEU": "L", "LYS":"K", "MET":"M", "PHE":"F", "PRO":"P",\
                            "SER":"S", "THR":"T", "TRP":"W", "TYR":"Y", "VAL":"V"}
def convert_to_one_letter(string):
    amino_acid = str(string[0:3])
    return convert_to_one_letter_dic[amino_acid]

### Set Palette

In [ ]:
palette = sns.color_palette("deep")

In [ ]:
blue = sns.color_palette("deep")[0]
orange = sns.color_palette("deep")[1]
green = sns.color_palette("deep")[2]
red = sns.color_palette("deep")[3]
purple = sns.color_palette("deep")[4]
brown = sns.color_palette("deep")[5]
pink = sns.color_palette("deep")[6]
grey = sns.color_palette("deep")[7]
gold = sns.color_palette("deep")[8]
turqoise = sns.color_palette("deep")[9]

### Define Universes

In [ ]:
# Read trajectory
u1 = mda.Universe("HlyB_traj_1_chainAB.pdb", 'HlyB_traj_1_nowater_skip100.xtc')
u2 = mda.Universe("HlyB_traj_2_chainAB.pdb", 'HlyB_traj_2_nowater_skip100.xtc')
u3 = mda.Universe("HlyB_traj_3_chainAB.pdb", 'HlyB_traj_3_nowater_skip100.xtc')

In [ ]:
labels = ["u1","u2","u3"]

In [ ]:
# Check length of the trajectories
print(len(u1.trajectory), len(u2.trajectory), len(u3.trajectory))

### Ensemble Cluster Analysis

This analysis only works with preselected atoms in trajectories. Only reduce number of frames to speed up calculation.

In [ ]:
# import modules
from MDAnalysis.analysis import encore
from MDAnalysis.analysis.encore.clustering import ClusteringMethod as clm

In [ ]:
# Cluster
ces0, details0 = encore.ces(ensembles=[u1,u2,u3], select="name CA")

In [ ]:
# Print cluster info
cluster_collection = details0['clustering'][0]
print(type(cluster_collection))
print('We have found {} clusters'.format(len(cluster_collection)))

In [ ]:
# k-means clustering
km1 = clm.KMeans(12,  # no. clusters
                 init = 'k-means++',  # default
                 algorithm="auto")    # default

km2 = clm.KMeans(9,  # no. clusters
                 init = 'k-means++',  # default
                 algorithm="auto")    # default

km3 = clm.KMeans(6,  # no. clusters
                 init = 'k-means++',  # default
                 algorithm="auto")    # default

km4 = clm.KMeans(3,  # no. clusters
                 init = 'k-means++',  # default
                 algorithm="auto")    # default



In [ ]:
# Plot clustering results
titles = ['Kmeans 12 clusters', 'Kmeans 9 clusters', 'Kmeans 6 clusters', 'Kmeans 3 clusters']
fig2, axes = plt.subplots(1, 4, sharey=True, figsize=(15, 4))
for i, (data, title) in enumerate(zip(ces2, titles)):
    imi = axes[i].imshow(data, vmax=np.log(2), vmin=0)
    axes[i].set_xticks(np.arange(3))
    axes[i].set_xticklabels(labels)
    axes[i].set_title(title)
plt.yticks(np.arange(3), labels)
cbar2 = fig2.colorbar(imi, ax=axes.ravel().tolist())
cbar2.set_label('Jensen-Shannon divergence')
plt.savefig("Ensemble Cluster K means All Trajectories.png", dpi = 300)

In [ ]:
#estimate errors
avgs, stds = encore.ces([u1, u2, u3],
                         select='name CA',
                         clustering_method=km4,
                         estimate_error=True,
                         ncores=4)

In [ ]:
avgs

In [ ]:
stds

## Protein-Substrate Contacts

### Import Data

In [ ]:
sub_df_1 = pd.read_csv("data/protein_sub_contacts_traj_1.csv")
sub_df_2 = pd.read_csv("data/protein_sub_contacts_traj_2.csv")
sub_df_3 = pd.read_csv("data/protein_sub_contacts_traj_3.csv")

In [ ]:
sub_frames = [sub_df_1,sub_df_2,sub_df_3]
df_substrate = pd.concat(sub_frames, ignore_index=True)

In [ ]:
df_substrate

### Add Annotations

In [ ]:
# get resids
df_substrate["Substrate Joint Position"] = df_substrate["Substrate Protein Type"].astype(str) + df_substrate["Substrate Protein ResID"].astype(str)
df_substrate["Acceptor Joint Position"] = df_substrate["Acceptor Protein Type"].astype(str) + df_substrate["Acceptor Protein ResID"].astype(str)

In [ ]:
 #sort values by protein position
df_substrate.sort_values("Acceptor Protein ResID", axis = 0, ascending = True, inplace = True)

In [ ]:
# calculate fraction frames for interactions
df_substrate["Fraction Frames"] = df_substrate["#Frames"]/7500

In [ ]:
# group data
group_sub_contact = df_substrate.groupby(["ID","Acceptor ChainID", "Acceptor Protein ResID","Acceptor Joint Position","Substrate Joint Position"])["Fraction Frames"].mean().unstack()

In [ ]:
#Reset index to remove multi-indexing
group_sub_contact.reset_index(inplace = True)

In [ ]:
#sort values by protein position
group_sub_contact.sort_values("Acceptor Protein ResID", axis = 0, ascending = True, inplace = True)

In [ ]:
# Apply melt
c_melt = group_sub_contact.melt(id_vars = ["ID","Acceptor Protein ResID", "Acceptor Joint Position", "Acceptor ChainID"], value_vars = ["D1","V2","K3","E4","E5","R6","T7",
                                                                                                   "A8","A9","S10","L11","L12","Q13",
                                                                                                   "L14","S15","G16","N17","A18","S19","D20", "F21",
                                                                                                   "S22","Y23"],
                                                                                                   value_name="Fraction_Frames")

##### Label amino acid type by colour - for later plotting

red - negatively charged\
blue - positively charged\
green - hydrophobic\
pink - polar\
yellow - special case (cysteine, glycine, proline)

In [ ]:
amino_colours = {"D1":"red","V2":"green","K3":"blue","E4":"red","E5":"red","R6":"blue","T7":"pink","A8":"green",\
                "A9":"green","S10": "pink","L11": "green","L12":"green","Q13": "pink","L14":"green","S15":"pink",\
                 "G16":"yellow","N17":"pink", "A18":"green","S19":"pink","D20":"red","F21":"green","S22":"pink",\
                 "Y23":"green"}
def apply_amino_colour(string):
    return str(amino_colours[string])

In [ ]:
amino_type = {"D1":"negative charge","V2":"hydrophobic","K3":"positive charge","E4":"negative charge",\
              "E5":"negative charge","R6":"positive charge","T7":"polar","A8":"hydrophobic","A9":"hydrophobic",\
              "S10": "polar","L11": "hydrophobic","L12":"hydrophobic","Q13": "polar","L14":"hydrophobic",\
              "S15":"polar","G16":"polar","N17":"polar", "A18":"hydrophobic","S19":"polar","D20":"negative charge",\
              "F21":"hydrophobic","S22":"polar","Y23":"hydrophobic"}
def apply_amino_type(string):
    return str(amino_type[string])

In [ ]:
# Apply colours
c_melt["Substrate Color Column"] = c_melt["Substrate Joint Position"].apply(apply_amino_colour)
c_melt["Substrate Type Column"] = c_melt["Substrate Joint Position"].apply(apply_amino_type)

In [ ]:
#remove nan values for comparing to lipid interactions/bokeh
c_melt_nonan = c_melt.dropna(inplace = False)

In [ ]:
# remove small values for plotting
c_melt_remove = c_melt[c_melt["Fraction_Frames"] >= 0.05]
c_melt_nonan_remove = c_melt_nonan[c_melt_nonan["Fraction_Frames"] >= 0.05]

In [ ]:
# sort again otherwise x axis won't be correct!
c_melt_nonan_remove.sort_values("Acceptor Protein ResID", axis = 0, ascending = True, inplace = True) # small values and nans removed
c_melt_remove.sort_values("Acceptor Protein ResID", axis = 0, ascending = True, inplace = True) # small values removed

### Exploratory Plots

In [ ]:
# Interactions by ID
chain_C_fig = sns.catplot(x="Acceptor Joint Position", y="Fraction_Frames", hue='ID', data=c_melt_remove, height = 5, aspect = 10).fig;  
chain_C_fig.savefig("Substrate Interactions by ID.png", dpi = 300)

In [ ]:
# Interactions by ID - boxplot
chain_C_fig = sns.catplot(x="Acceptor Joint Position", y="Fraction_Frames", hue='ID', kind='box', data=c_melt_remove, height = 5, aspect = 15).fig;  
chain_C_fig.savefig("Substrate Interactions by ID boxplot.png", dpi = 300)

In [ ]:
# Interactions by sub type
chain_C_fig_type = sns.catplot(x="Acceptor Joint Position", y="Fraction_Frames", hue='Substrate Type Column', data=c_melt_remove, height = 5, aspect = 10).fig;  
chain_C_fig_type.savefig("Substrate Interactions by amino acid type.png", dpi = 300)

In [ ]:
# Interactions by sub type - boxplot
chain_C_fig_type = sns.catplot(x="Acceptor Joint Position", y="Fraction_Frames", hue='Substrate Type Column', kind='box', data=c_melt_remove, height = 5, aspect = 15).fig;  
chain_C_fig_type.savefig("Substrate Interactions by amino acid type boxplot.png", dpi = 300)

In [ ]:
# add order
chain_C_fig_type = sns.catplot(x="Acceptor Joint Position", y="Fraction_Frames", 
                               hue='Substrate Type Column', kind='box', data=c_melt_remove,
                               order=['S3','C4','H5','K6','I7','Y9','T39','G40','L41','G42','L43',
                                     'T44','K61','W77','E79','D80','G81','H83','S245','R247','G249',
                                     'D250','V252','A253','R256','F323','N326','A327','D328','Q330',
                                     'S331','V334','E335','R443','K478','P479','D480','S481','P482',
                                     'V483','L555','N556','R557','F587','E590','L591','R592','E593',
                                     'N596','T597','I598','V599','G600','E601','Q602','G605','E639',
                                     'K678'],
                               height = 5, aspect = 15).fig;   
chain_C_fig_type.savefig("Substrate Interactions by amino acid type boxplot ordered correct.png", dpi = 300)

### Plots for Thesis

In [ ]:
# set palette dic
palette_dic = {"negative charge":blue, "positive charge":orange,"polar":green,"hydrophobic":red}

In [ ]:
# use palette for correct colours
chain_C_fig_type = sns.catplot(x="Acceptor Joint Position", y="Fraction_Frames", 
                               hue='Substrate Type Column', kind='box', data=c_melt_remove,
                               order=['S3','C4','H5','K6','I7','Y9','T39','G40','L41','G42','L43',
                                     'T44','K61','W77','E79','D80','G81','H83','S245','R247','G249',
                                     'D250','V252','A253','R256','F323','N326','A327','D328','Q330',
                                     'S331','V334','E335','R443','K478','P479','D480','S481','P482',
                                     'V483','L555','N556','R557','F587','E590','L591','R592','E593',
                                     'N596','T597','I598','V599','G600','E601','Q602','G605','E639',
                                     'K678'],
                               palette=palette_dic,
                               height = 5, aspect = 15).fig;
plt.xlabel('Protein Residue')
plt.ylabel('Fraction Frames')
chain_C_fig_type.savefig("Substrate Interactions by amino acid type boxplot ordered.png", dpi = 300)

In [ ]:
# show chains only
chain_C_fig_type = sns.catplot(x="Acceptor Joint Position", y="Fraction_Frames", 
                               hue='Acceptor ChainID', kind='box', data=c_melt_remove,
                               order=['S3','C4','H5','K6','I7','Y9','T39','G40','L41','G42','L43',
                                     'T44','K61','W77','E79','D80','G81','H83','S245','R247','G249',
                                     'D250','V252','A253','R256','F323','N326','A327','D328','Q330',
                                     'S331','V334','E335','R443','K478','P479','D480','S481','P482',
                                     'V483','L555','N556','R557','F587','E590','L591','R592','E593',
                                     'N596','T597','I598','V599','G600','E601','Q602','G605','E639',
                                     'K678'],
                               height = 5, aspect = 15).fig;
plt.xlabel('Protein Residue')
plt.ylabel('Fraction Frames')
chain_C_fig_type.savefig("Substrate Interactions by chain boxplot ordered.png", dpi = 300)

In [ ]:
# assign trajectories
c_melt_remove = c_melt_remove.assign(Trajectory=c_melt_remove.ID.map({'HlyB Closed Atomistic': "u1", 'HlyB Closed Atomistic Repeat': "u2", 'HlyB Closed Atomistic Repeat 2': 'u3'}))

In [ ]:
# plot by ID
chain_C_fig_type = sns.catplot(x="Acceptor Joint Position", y="Fraction_Frames", 
                               hue='ID', kind='box', data=c_melt_remove,
                               order=['S3','C4','H5','K6','I7','Y9','T39','G40','L41','G42','L43',
                                     'T44','K61','W77','E79','D80','G81','H83','S245','R247','G249',
                                     'D250','V252','A253','R256','F323','N326','A327','D328','Q330',
                                     'S331','V334','E335','R443','K478','P479','D480','S481','P482',
                                     'V483','L555','N556','R557','F587','E590','L591','R592','E593',
                                     'N596','T597','I598','V599','G600','E601','Q602','G605','E639',
                                     'K678'],
                               height = 5, aspect = 15).fig;
chain_C_fig_type.savefig("Substrate Interactions by ID boxplot ordered rename.png", dpi = 300)

In [ ]:
# plot by trajectory
chain_C_fig_type = sns.catplot(x="Acceptor Joint Position", y="Fraction_Frames", 
                               hue='Trajectory', kind='box', data=c_melt_remove,
                               order=['K6','Y9','T39','G40','L41','G42','L43','E79','D80',
                                      'G81','R247','G249','A253','R256','F323','A327','Q330',
                                     'S331','V334','N556','R557','R592','I598','E601'],
                               height = 5, aspect = 3).fig; 
plt.xlabel('ResID')
plt.ylabel('Fraction Frames')
chain_C_fig_type.savefig("Consensus Substrate Interactions by trajectory boxplot correct.png", dpi = 300)

In [ ]:
# plot by interaction consensus
chain_C_fig_type = sns.catplot(x="Acceptor Joint Position", y="Fraction_Frames", 
                               hue='Acceptor ChainID', kind='box', data=c_melt_remove,
                               order=['K6','Y9','T39','G40','L41','G42','L43','E79','D80',
                                      'G81','R247','G249','A253','R256','F323','A327','Q330',
                                     'S331','V334','N556','R557','R592','I598','E601'],
                               height = 5, aspect = 3).fig;  
plt.xlabel('ResID')
plt.ylabel('Fraction Frames')
chain_C_fig_type.savefig("Consensus Substrate Interactions by chain boxplot ordered correct.png", dpi = 300)

In [ ]:
# plot by interaction consensus
chain_C_fig_type = sns.catplot(x="Acceptor Joint Position", y="Fraction_Frames", 
                               hue='Substrate Type Column', kind='box', data=c_melt_remove,
                               order=['K6','Y9','T39','G40','L41','G42','L43','E79','D80',
                                      'G81','R247','G249','A253','R256','F323','A327','Q330',
                                     'S331','V334','N556','R557','R592','I598','E601'],
                               palette=palette_dic,
                               height = 5, aspect = 3).fig; 
plt.xlabel('ResID')
plt.ylabel('Fraction Frames')
chain_C_fig_type.savefig("Consensus Substrate Interactions by sub type boxplot ordered correct.png", dpi = 300)

### Bokeh Plot

In [ ]:
from bokeh.plotting import Figure, output_notebook, show, save
from bokeh.models import ColumnDataSource, HoverTool, GroupFilter, CDSView
from bokeh.models import BoxSelectTool

output_notebook()

In [ ]:
c_data = ColumnDataSource(c_melt_nonan_remove)

c_int = Figure(tools='pan,wheel_zoom,box_zoom,reset', tooltips=[('ID','@{ID}'),('Chain', '@{Acceptor ChainID}'),('Sub', '@{Substrate Joint Position}'), ('Acceptor', '@{Acceptor Joint Position}')], width=2000, height=300, x_axis_label = "Acceptor Position", y_axis_label = "Fraction Frames")
c_int.scatter(source=c_data, x='Acceptor Protein ResID', y='Fraction_Frames', \
                  fill_color='Substrate Color Column', size=5)
c_int.add_tools(BoxSelectTool(dimensions="width"))

show(c_int);

##### Save Bokeh Plot

In [ ]:
save(c_int, filename='data/Substrate Interactions Bokeh', resources='inline', title='Substrate Interactions');